# Analysis of the Migration Range of the American Redstart (Setophaga ruticilla) from 2004 to 2024


### Introduction
The American Redstart (Setophaga ruticilla) is a colorful warbler that migrates across the Americas each year. Males have a dark black head and chest, with bright orange on the wings, sides, and tail, while females are a lighter grey color with yellow highlights. These warblers prefer nesting in open woodlands across North America [(Cornell Lab of Ornithology)](https://www.allaboutbirds.org/guide/American_Redstart/overview). They spend the winters in Central and South America, and migrate as far as Northern Canada for breeding during the summer. Like many migratory birds, climate change and human developments are impacting the Redstart's migration patterns and survival according to the [Cary Institute of Ecosystem Studies](https://www.caryinstitute.org/news-insights/feature/climate-change-pushing-american-redstarts-breeding-range-southward).


<img src="https://www.allaboutbirds.org/guide/assets/og/75237001-1200px.jpg" width="600">



### Migration Patterns, Climate Change, and Deforestation
American Redstarts are being affected by climate change in some parts of their migration range. According to Bryant Dossman, a conservation and migration ecology scientist, Redstarts are particularly affected by increasingly dry conditions in the tropics of South America and Jamaica where many Redstarts spend the winter [(Heisman, 2024)](https://www.allaboutbirds.org/news/american-redstarts-can-speed-up-their-migration-but-theres-a-cost/). There is strong evidence that increasingly dry conditions in the tropics are linked to climate change and deforestation [(Y. Malhi, et al)](https://royalsocietypublishing.org/rstb/article-abstract/359/1443/331/20382/Tropical-forests-and-the-global-carbon-cycle?redirectedFrom=fulltext) and [(Franco. M.A, et al)](https://doi.org/10.1038/s41467-025-63156-0). According to Dossman, these conditions are making it harder for the Redstart to find enough food and insects to put on enough weight for the migration journey north. Dossman observes that this causes the Redstart to often leave later in the spring. Dossman used GPS tracking devices on the birds to study their migration speed. He found that bird that left late migrated faster to make up for lost time. However, he notes that migrating faster increases stress and energy expenditure of the birds, making them more likely to die on the journey. Dossman found that birds that left late had a 6% increased risk of death on their migration journey as opposed to birds that left on time [(Heisman, 2024)](https://www.allaboutbirds.org/news/american-redstarts-can-speed-up-their-migration-but-theres-a-cost/). Another similar study from 2015 found very similar results to Dossman [(Cooper, N.W, et al)](https://doi.org/10.1890/14-1365.1). 

These studies indicate that an increasing lack of resources in South America is impacting the Redstart's range. According to the Cary Institute of Ecosystem Studies, the Redstart is migrating farther south into South America during the winters in search of food, and not as far north into Canada during the summers because of the delay migrating north. The overall effect is that the Redstart's range is shifting southwards [(Cary Institute of Ecosystem Studies)](https://www.caryinstitute.org/news-insights/feature/climate-change-pushing-american-redstarts-breeding-range-southward).


### Migration Mapping

Because of the impacts described above, knowing the month-by-month range of the Redstart's migration path can be a useful tool for conservation efforts. In addition to the month-by-month range, assessing the range over years and decades can confirm important information about how climate change and deforestation is impacting the Redstart's range. 

### Migration Data from GBIF

To map the Redstart's annual migration, I will be using occurrence data from the [Global Biodiversity Information Facility (GBIF)](https://www.gbif.org/). Occurrence data is largely collected using [eBird](https://ebird.org/home) and [iNaturalist](https://www.inaturalist.org/), as well as several other reputable citizen science data sources [(GBIF Data Processing)](https://www.gbif.org/data-processing).

### Ecoregion Data

I will be plotting occurrence (sightings of Redstarts) data within ecoregions to visualize the Redstart migration on a map. For the ecoregion data, I will be using data published in 2017 by [RESOLVE](https://www.resolve.ngo/) and [OneEarth](https://www.oneearth.org/announcing-the-release-of-ecoregion-snapshots/).

## Normalization Methods

The goal of this analysis and migration map making process is to not only represent which ecoregions had occurrences of Redstarts during a given year, but to also represent the density of observations. However, representing density can be difficult because not all ecoregions are the same size, and some ecoregions may have more people in them leading to more observations. To account for this, I will be normalizing the data by area, and also by space and time. 

#### Normalization by Area
To normalize by area, I divide the number of occurrences in an ecoregion by the area of that ecoregion. Not all ecoregions are the same size, so this step accounts for this. This gives us count of the number of occurences for a given unit area.

#### Normalization by Space and Time
The goal of this step is to remove systematic differences such as some months having more sampling efforts because more birdwatchers are outside, or some regions are more heavily surveyed due to a higher human population.

### Python workflow
Below is the Python workflow I'm using to complete this analysis. In addition to this notebook, I'm using separate Juypter notebooks for each GBIF download for the specific years of interest. **Important: The download notebooks should be run first.** This notebook depends on the data produced in the download notebooks. 

In [1]:
# Import packages
import os # For file paths, data management, interacting with the operating system
import pathlib # Object oriented file path management
import time # Time how long operations take in Python
import zipfile # Deal with zip files
from getpass import getpass # Security login for downloading GBIF data
from glob import glob # Search for specific file extensions

# Libraries for data analysis 
import pandas as pd # Data analysis for tabular data
import geopandas as gpd # Data analysis for geospatial data
import pygbif.occurrences as occ # Get occurrence data
import pygbif.species as species # Get species data
import requests # Download data

# Libraries for Dynamic mapping
import cartopy.crs as ccrs #Coordinate reference systems 
import panel as pn #customize the dynamic map
import hvplot.pandas #make the holoview plot with geo-dataframe
import calendar # Change month numbers to month names

## Migration Maps for years 2004, 2009, 2014, 2019, and 2024
**Important:** This notebook assumes you have already run all 5 of the download notebooks to download, process, and map the migration data for the years of interest. In the code chunk below, we will be accessing these maps via .geojson files that were saved/created at the end of each of the download notebooks. These .geojson files should be in your main working directory and readily accessible to this notebook. This notebook simply runs the steps necessary to join these individual maps into a new combined map for all the years of interest.

In [2]:
# Preparing data to make half-decade migration map

# list of years to be used
years = [2004, 2009, 2014, 2019, 2024]

# empty list of gdfs for the years of use
gdfs = []

# looping through the list of years and appending as geojson
for y in years:
    gdf = gpd.read_file(f"redstart_occurrence_{y}.geojson")  
    gdf["year"] = y
    gdfs.append(gdf)

combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

## Set the extent of the map
I manually set the extent so that the map fits nicely for this particular data. 

In [3]:
# Manually set extent of the map
xmin = -35000000
xmax = -2000000.0
ymax = 11000000.0
ymin = -4000000.0

Create a year-to-year slider for the map.

In [14]:
# Instead of a month-by-month slider, we want a year-to-year slider
# This step is for the hvplot

# Using the list of years we created earlier
year_slider = pn.widgets.DiscreteSlider(
    name='Year',
    options={str(y): y for y in years}
)

# Instead of using the .calendar package, we're using the year list
# I used ChatGPT for help with the "options=" syntax

Combine the data for all years into a new geodataframe, fix potential hvplot errors, and project to the Mercator Projection system to match the basemap tiles. 

In [ ]:
# Combine gdfs for all years into new geodataframe (combined_gdf)
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Check if combined_gdf has a CRS, and if not, set one (ESPG 4326)
if combined_gdf.crs is None:
    combined_gdf.set_crs("EPSG:4326", inplace=True)

# Fix invalid geometries. Otherwise the hvplot might not work
combined_gdf["geometry"] = combined_gdf.buffer(0)

# Project to Mercator to match the basemap tiles
combined_gdf = combined_gdf.to_crs("EPSG:3395")

# Remove empty geometries to prevent hvplot errors
combined_gdf = combined_gdf[~combined_gdf.geometry.is_empty]

# I used Chatgpt for parts of this workflow to help diagnose errors I was encountering with hvplot

### Yearly density calculation and representation
The previous maps were month-by-month, so these need to be combined into yearly maps, showing all the ecoregions for one year at once. Additionally, the density of the monthly observations need to be summed to show the annual density of observations per ecoregion. Taking the sum of "norm_occurrences" for each year will represent the density of observations per year. 

In [15]:
# Combine monthly rows into one row per ecoregion per year
combined_yearly = (
    combined_gdf
    .dissolve(
        by=["ecoregion_id", "name", "year"],   # Group polygons by year and keep attributes
        aggfunc={"norm_occurrences": "sum"}    # Take the sum of norm_occurrences per year
    )
    .reset_index()
)

Plot the result using the hvplot package.

In [11]:
# Create the interative plot using hvplot

combined_yearly = combined_yearly.to_crs(epsg=3857)

migration_plot_years = combined_yearly.hvplot(
    c="norm_occurrences",
    geo=True,
    crs=ccrs.Mercator(),
    tiles="CartoLight",
    groupby="year",
    xlim=(xmin, xmax),
    ylim=(ymin, ymax),
    frame_height=500,
    widget_location="bottom",
    title="American Redstart Range by Year"
)

#Display the plot
migration_plot_years


BokehModel(combine_events=True, render_bundle={'docs_json': {'004ae23d-c2b4-4af4-8c8b-cb61833b1d4c': {'version…

In [12]:
# Save the plot
migration_plot_years.save("redstart-migration-years.html", embed=True)

### Analysis of Results
The map above shows the entire migration range of the Redstart for the years of 2004, 2009, 2014, 2019, and 2024. In 2004, notice the northern extent of the Redstart in Northwestern Canada. This ecoregion disappears in later years, meaning the Redstart has not been observed in that northern ecoregion in more than a decade according to the GBIF data. As you scroll through the years, also notice the range extent in South America. In 2004, the range is fairly limited in South America. However, the Redstart's range in South America has gradually been increasing southward, as well as the density of observations. Notice that the density of observations in 2004 is relatively low. However, in more recent years, the density is much higher. 


### Conclusion
According to the [Cornell Lab of Ornithology](https://www.allaboutbirds.org/guide/American_Redstart/overview), the [Cary Institute of Ecosystem Studies](https://www.caryinstitute.org/news-insights/feature/climate-change-pushing-american-redstarts-breeding-range-southward), and the study by [Cooper et al](https://doi.org/10.1890/14-1365.1), the American Redstart's migration range has been forced southwards in recent decades due to climate change impacts on their habitat, particularly in Central and South America. This analysis focused on two decades of data of GBIF observations of the Redstart's range in order to observe trends in the extent of the Redstart's range. The resulting map shows an increase in the extent of the Redstart's southern range during the winter months, indicating that the bird is venturing farther into South America in search of food. Additionally, the Northern range reduced slightly. These findings are consistent with the studies cited above, showing that the Redstart's range is being affected by climate change and deforestation. While the species appears to be adapting and its population is stable, these affects are putting an increased strain on its migration patterns and could lead to more negative impacts in the future if conditions worsen [(Heisman, 2024)](https://www.allaboutbirds.org/news/american-redstarts-can-speed-up-their-migration-but-theres-a-cost/).

### Works Cited


Cary Institute of Ecosystem Studies. Climate change is pushing the American redstart’s breeding range southward. https://www.caryinstitute.org/news-insights/feature/climate-change-pushing-american-redstarts-breeding-range-southward


Cooper, N.W., Sherry, T.W. and Marra, P.P. (2015), Experimental reduction of winter food decreases body condition and delays migration in a long-distance migratory bird. Ecology, 96: 1933-1942. https://doi.org/10.1890/14-1365.1 


Cornell Lab of Ornithology. American Redstart. https://www.allaboutbirds.org/guide/American_Redstart/overview 


Franco, M.A., Rizzo, L.V., Teixeira, M.J. et al. How climate change and deforestation interact in the transformation of the Amazon rainforest. Nat Commun 16, 7944 (2025). https://doi.org/10.1038/s41467-025-63156-0


Heisman, Rebecca. American Redstarts Can Speed Up Their Migration, but There’s a Cost. Cornell Lab of Ornithology. 2024. https://www.allaboutbirds.org/news/american-redstarts-can-speed-up-their-migration-but-theres-a-cost/.


Y. Malhi, O. L. Phillips, Wolfgang Cramer, Alberte Bondeau, Sibyll Schaphoff, Wolfgang Lucht, Benjamin Smith, Stephen Sitch; Tropical forests and the global carbon cycle: impacts of atmospheric carbon dioxide, climate change and rate of deforestation. Philos Trans R Soc Lond B Biol Sci 29 March 2004; 359 (1443): 331–343. https://doi.org/10.1098/rstb.2003.1428


